# Fine-Tune de um LLM para sumarização de diálogo

Neste notebook, você ajustará um LLM existente do Hugging Face para um resumo aprimorado do diálogo. Você usará o modelo [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5), que fornece um modelo ajustado para instruções de alta qualidade e pode resumir o texto imediatamente. 

# Índice

- [ 1 - Carregar dependências necessárias, conjunto de dados e LLM](#1)
  - [ 1.1 - Carregar conjunto de dados e LLM](#1.1)
  - [ 1.2 - Teste o modelo com inferência zero shot](#1.2)
- [ 2 - Execute o ajuste fino completo](#2)
  - [ 2.1 - Pré-processar o conjunto de dados de resumo de diálogo](#2.1)
  - [ 2.2 - Ajuste o modelo com o conjunto de dados pré-processado](#2.2)
  - [ 2.3 - Avalie o modelo qualitativamente (avaliação humana)](#2.3)
  - [ 2.4 - Avalie o modelo quantitativamente (com a métrica ROUGE)](#2.4)
- [ 3 - Execute o ajuste fino eficiente de parâmetros (PEFT)](#3)
  - [ 3.1 - Configure o modelo PEFT/LoRA para ajuste fino](#3.1)
  - [ 3.2 - Treinando o adaptador PEFT](#3.2)
  - [ 3.3 - Avaliar o modelo qualitativamente (avaliação humana)](#3.3)
  - [ 3.4 - Avalie o modelo quantitativamente (com a métrica ROUGE)](#3.4)

<a name='1'></a>
## 1 - Carregar dependências necessárias, conjunto de dados e LLM

<a name='1.1'></a>
### 1.1 - Carregar dependências necessárias

In [ ]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

Importe os componentes necessários. 

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

<a name='1.1'></a>
### 1.2 - Carregar conjunto de dados e LLM

Você continuará experimentando o conjunto de dados [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) Hugging Face. Ele contém mais de 10.000 diálogos com resumos e tópicos rotulados manualmente.

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

Carregue o [modelo FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) pré-treinado e seu tokenizer diretamente do HuggingFace. Observe que você usará a [versão pequena](https://huggingface.co/google/flan-t5-base) do FLAN-T5. A configuração `torch_dtype=torch.bfloat16` especifica o tipo de memória a ser usada por este modelo.

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

É possível extrair a quantidade de parâmetros do modelo e descobrir quantos deles são treináveis. A função a seguir pode ser usada para fazer isso.

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

<a name='1.2'></a>
### 1.2 - Teste o modelo com inferência zero shot

Teste o modelo com a inferência zero hhot. Você pode ver que o modelo tem dificuldade para resumir o diálogo em comparação com o resumo feito por um humano, mas extrai algumas informações importantes do texto que indicam que o modelo pode ser ajustado para a tarefa em questão.

In [ ]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'PROMPT DE ENTRADA:\n{prompt}')
print(dash_line)
print(f'SUMÁRIO HUMANO:\n{summary}\n')
print(dash_line)
print(f'GERADO PELO MODELO - ZERO SHOT:\n{output}')

<a name='2'></a>
## 2 - Execute o ajuste fino completo

<a name='2.1'></a>
### 2.1 - Pré-processar o conjunto de dados de resumo de diálogo

Você precisa converter os pares diálogo-resumo (prompt-completion) em instruções explícitas para o LLM:

Explicação

Prompt de treinamento (diálogo):
```
Summarize the following conversation.

    Palla: This is his part of the conversation.
    Ana: This is her part of the conversation.
    
Summary: 
```

Resposta do treinamento (resumo):
```
Both Palla and Ana participated in the conversation.
```

Em seguida, pré-processe o conjunto de dados de prompt-completion em tokens e extraia seus `input_ids`

*Explicação*: A função abaixo está tokenizando as amostras do conjunto de dados para prepará-los para modelagem de sequência usando um modelo de transformer.

A função ```tokenize_function``` pega cada amostra, que contém um diálogo e seu respectivo resumo. Ela cria um prompt concatenando o prompt de início (instrução), o diálogo e o completion. Este prompt é então tokenizado e os IDs de saída são atribuídos ao campo ```input_ids``` da amostra.

A função também tokeniza separadamente o resumo e atribui esses IDs ao campo ```labels```.

Ela está mapeando todo o conjunto de dados para tokenizar todos as amostras. O argumento ```batched=True`````` significa que ele processará as amostras em lotes para maior eficiência.

Após mapear o conjunto de dados, ele remove os campos de texto puro que foram usados para tokenização ('id', 'topic', 'dialog', 'summary').

O resultado final é um conjunto de dados tokenizado onde cada amostra contém o prompt de entrada tokenizado em ```input_ids``` e o resumo tokenizado em ```labels```. Este formato pode então ser usado para ajustar um modelo, permitindo prever os ```labels``` dados os ```input_ids```.

In [ ]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Para economizar algum tempo, você utilizará uma subamostra do conjunto de dados:

*Explicação*: O que estamos fazendo aqui é reduzir o ```tokenized_datasets``` para 1%, mantendo apenas a 100ª amostra com base no índice, ao mesmo tempo que preservamos os índices originais no conjunto de dados filtrado.

In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Verifique as formas de todas as três partes do conjunto de dados:

*Explicação*: O código abaixo imprime informações sobre as formas e tamanhos de diferentes conjuntos de dados tokenizados.

A propriedade .shape retorna uma tupla que descreve o número de elementos em cada dimensão do conjunto de dados. Essas instruções revelam quantas amostras/linhas e features/colunas estão em cada divisão dos dados tokenizados.

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

O conjunto de dados de saída está pronto para ajuste fino.

<a name='2.2'></a>
### 2.2 - Ajuste o modelo com o conjunto de dados pré-processado

Agora utilize a classe Hugging Face `Trainer` integrada (veja a documentação [aqui](https://huggingface.co/docs/transformers/main_classes/trainer)). Passe o conjunto de dados pré-processado com referência ao modelo original. 

In [ ]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

Treinar uma versão totalmente ajustada do modelo levaria algumas horas em uma GPU. Para economizar tempo, baixe um ponto de verificação do modelo totalmente ajustado para usar no restante deste notebook. Esse modelo totalmente ajustado também será chamado de **modelo de instrução**.

In [ ]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint/

O tamanho do modelo de instrução baixado é de aproximadamente 1 GB.

In [ ]:
!ls -alh ./flan-dialogue-summary-checkpoint/pytorch_model.bin

Crie uma instância da classe `AutoModelForSeq2SeqLM` para o modelo de instrução:

AutoModelForSeq2SeqLM é uma classe da biblioteca Transformers que representa um modelo pré-treinado de ```sequence-to-sequence``` para modelagem de linguagem.

Este código carrega o modelo de resumo de diálogo pré-treinado armazenado no caminho do ponto de verificação, em um formato ```bfloat16``` para maior eficiência. O modelo carregado é atribuído à variável ```instruct_model```, que sera usada para gerar resumos de diálogos com base no modelo ajustado.

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype=torch.bfloat16)

<a name='2.3'></a>
### 2.3 - Avalie o modelo qualitativamente (avaliação humana)

Tal como acontece com muitos aplicativos GenAI, uma abordagem qualitativa em que você se pergunta "Meu modelo está se comportando da maneira que deveria?" geralmente é um bom ponto de partida. No exemplo abaixo (o mesmo com o qual iniciamos este notebook), você pode ver como o modelo ajustado é capaz de criar um resumo razoável do diálogo em comparação com a incapacidade original de entender o que está sendo pedido ao modelo.

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'SUMÁRIO HUMANO:\n{human_baseline_summary}')
print(dash_line)
print(f'MODELO ORIGINAL:\n{original_model_text_output}')
print(dash_line)
print(f'MODELO DE INSTRUÇÃO:\n{instruct_model_text_output}')

<a nome='2.4'></a>
### 2.4 - Avalie o modelo quantitativamente (com a métrica ROUGE)

A [métrica ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) ajuda a quantificar a validade dos resumos produzidos pelos modelos. Ela compara os resumos a um resumo "base" que geralmente é criado por um ser humano. Embora não seja perfeito, indica a eficácia da sumarização que conseguimos através do ajuste fino.

In [ ]:
rouge = evaluate.load('rouge')

Gerando as saídas para a amostra do conjunto de dados de teste (apenas 10 diálogos e resumos para economizar tempo) e salvando os resultados.

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)
    
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

Avaliando os modelos calculando métricas ROUGE. Observe a melhora nos resultados!

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('MODELO ORIGINAL:')
print(original_model_results)
print('MODELO DE INSTRUÇÃO:')
print(instruct_model_results)

O arquivo `data/dialogue-summary-training-results.csv` contém uma lista pré-preenchida de todos os resultados do modelo que você pode usar para avaliar uma seção maior de dados. Vamos fazer isso para cada um dos modelos:

In [ ]:
results = pd.read_csv("data/dialogue-summary-training-results.csv")

human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('MODELO ORIGINAL:')
print(original_model_results)
print('MODELO DE INSTRUÇÃO:')
print(instruct_model_results)

Os resultados mostram uma melhoria substancial em todas as métricas ROUGE:

In [ ]:
print("Melhoria percentual absoluta do MODELO DE INSTRUÇÃO em relação ao sumários GERADOS POR HUMANOS")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

<a nome='3'></a>
## 3 - Execute o ajuste fino eficiente de parâmetros (PEFT)

Agora, vamos realizar o **Parameter Efficient Fine-Tuning (PEFT)** em vez do "ajuste fino completo" como você fez acima. PEFT é uma forma de ajuste fino de instrução que é muito mais eficiente do que o ajuste fino completo - com resultados de avaliação comparáveis, como você verá em breve.

PEFT é um termo genérico que inclui **Low-Rank Adaptation (LoRA)** e prompt tuning (que NÃO É O MESMO que prompt engineering!). Na maioria dos casos, quando alguém diz PEFT, normalmente quer dizer LoRA. LoRA, em um nível muito alto, permite ao usuário ajustar seu modelo usando menos recursos computacionais (em alguns casos, uma única GPU). Após o ajuste fino para uma tarefa, caso de uso ou tenant específico com LoRA, o resultado é que o LLM original permanece inalterado e surge um “adaptador LoRA” recém-treinado. Este adaptador LoRA é muito, muito menor que o LLM original - da ordem de um dígito percentual do tamanho original do LLM (MBs vs GBs).

Dito isto, no momento da inferência, o adaptador LoRA precisa ser reunido e combinado com seu LLM original para atender à solicitação de inferência. O benefício, entretanto, é que muitos adaptadores LoRA podem reutilizar o LLM original, o que reduz os requisitos gerais de memória ao atender a múltiplas tarefas e casos de uso.

<a nome='3.1'></a>
### 3.1 - Configure o modelo PEFT/LoRA para ajuste fino

Você precisa configurar o modelo PEFT/LoRA para ajuste fino com um novo adaptador de camada/parâmetro. Usando PEFT/LoRA, você congela o LLM subjacente e treina apenas o adaptador. Dê uma olhada na configuração LoRA abaixo. Observe o hiperparâmetro rank (`r`), que define o rank/dimensão do adaptador a ser treinado.

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Adicione camadas/parâmetros do adaptador LoRA ao LLM original a ser treinado.

In [ ]:
peft_model = get_peft_model(original_model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

<a nome='3.2'></a>
### 3.2 - Treinando o adaptador PEFT

Defina argumentos de treinamento e crie uma instância `Trainer`.

In [ ]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1    
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [ ]:
peft_trainer.train()

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Esse treinamento foi realizado em um subconjunto de dados. Para carregar um modelo PEFT totalmente treinado, vamos ler um ponto de verificação de um modelo PEFT do S3.

In [ ]:
!aws s3 cp --recursive s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/ ./peft-dialogue-summary-checkpoint-from-s3/ 

Verifique se o tamanho deste modelo é bem menor que o LLM original:

In [ ]:
!ls -al ./peft-dialogue-summary-checkpoint-from-s3/adapter_model.bin

Preparando este modelo adicionando um adaptador ao modelo FLAN-T5 original. Você está definindo `is_trainable=False` porque o plano é apenas realizar inferência com este modelo PEFT. Se você estivesse preparando o modelo para treinamento adicional, você definiria `is_trainable=True`.

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       './peft-dialogue-summary-checkpoint-from-s3/', 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

O número de parâmetros treináveis será `0` devido à configuração `is_trainable=False`:

In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

<a nome='3.3'></a>
### 3.3 - Avaliar o Modelo Qualitativamente (Avaliação Humana)

Faça inferências para o mesmo exemplo das seções [1.3](#1.3) e [2.3](#2.3), com o modelo original, totalmente ajustado e modelo PEFT.

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
baseline_human_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'SUMÁRIO GERADO POR HUMANO:\n{human_baseline_summary}')
print(dash_line)
print(f'MODELO ORIGINAL:\n{original_model_text_output}')
print(dash_line)
print(f'MODELO DE INSTRUÇÃO:\n{instruct_model_text_output}')
print(dash_line)
print(f'MODELO PEFT: {peft_model_text_output}')

<a nome='3.4'></a>
### 3.4 - Avalie o modelo quantitativamente (com métrica ROUGE)
Realize inferências para a amostra do conjunto de dados de teste (apenas 10 diálogos e resumos para economizar tempo).

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]
    
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

Calculando a pontuação ROUGE para este subconjunto de dados.

In [ ]:

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('MODELO ORIGINAL:')
print(original_model_results)
print('MODELO DE INSTRUÇÃO:')
print(instruct_model_results)
print('MODELO PEFT:')
print(peft_model_results)

Observe que os resultados do modelo PEFT não são tão ruins, enquanto o processo de treinamento foi muito mais fácil!

Já calculamos a pontuação ROUGE no conjunto de dados completo, depois de carregar os resultados do arquivo `data/dialogue-summary-training-results.csv`. Agora vamos calcular os valores do modelo PEFT e verificar seu desempenho em comparação com outros modelos.

In [ ]:
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values
peft_model_summaries     = results['peft_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('MODELO ORIGINAL:')
print(original_model_results)
print('MODELO DE INSTRUÇÃO:')
print(instruct_model_results)
print('MODELO PEFT:')
print(peft_model_results)

Os resultados mostram menos melhorias em relação ao ajuste fino completo, mas os benefícios do PEFT normalmente superam as métricas de desempenho ligeiramente inferiores.

Calculando a melhoria do PEFT em relação ao modelo original:

In [ ]:
print("Melhoria percentual absoluta do MODELO PEFT em relação aos sumários GERADOS POR HUMANOS")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Agora calcule a melhoria do PEFT em relação a um modelo totalmente ajustado:

In [ ]:
print("Melhoria percentual absoluta do MODELO PEFT em relação ao MODELO DE INSTRUÇÃO")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Aqui você vê uma pequena diminuição percentual nas métricas ROUGE em comparação com o ajuste fino total. No entanto, o treinamento requer muito menos recursos de computação e memória (geralmente apenas uma GPU).